<a href="https://colab.research.google.com/github/abhranil-datascience/RegressionPOC/blob/master/MultivariateLinearRegressionPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/ML/Regression/MultivariateLinearRegression')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
########## Import Dataset ###############
import pandas as pd
dataset=pd.read_csv('50_Startups.csv')
X=dataset.iloc[:,0:4].values
Y=dataset.iloc[:,4].values
########## Apply One hot encoding to State ##############
XEncoded=X
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
XEncoded[:,3]=le.fit_transform(XEncoded[:,3])
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(categorical_features = [3])
XEncoded=ohe.fit_transform(XEncoded).toarray()
XEncoded=XEncoded[:,1:]
########## Feature Scaling ###############
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_Y=StandardScaler()
XScaled=sc_X.fit_transform(XEncoded)
YScaled=sc_Y.fit_transform(Y.reshape(-1,1))
######## Split Test Train data ###########
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,YScaled,test_size=0.1,random_state=42)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
######## Check P_Value ###############
import statsmodels.api as sm
import numpy as np
XTrainPVal=np.append(np.ones([XTrain.shape[0],1]),values=XTrain,axis=1)
XTestPVal=np.append(np.ones([XTest.shape[0],1]),values=XTest,axis=1)
#XTrainPValue=XTrainPVal[:,[0,1,2,3,4,5]]
#XTestPValue=XTestPVal[:,[0,1,2,3,4,5]]
#regressorOLS=sm.OLS(exog=XTrainPValue,endog=YTrain).fit()
#regressorOLS.summary()#Adj. R-squared:	0.948
#XTrainPValue=XTrainPVal[:,[0,2,3,4,5]]
#XTestPValue=XTestPVal[:,[0,2,3,4,5]]
#regressorOLS=sm.OLS(exog=XTrainPValue,endog=YTrain).fit()
#regressorOLS.summary()#Adj. R-squared:	0.949
#XTrainPValue=XTrainPVal[:,[0,3,4,5]]
#XTestPValue=XTestPVal[:,[0,3,4,5]]
#regressorOLS=sm.OLS(exog=XTrainPValue,endog=YTrain).fit()
#regressorOLS.summary()#Adj. R-squared:	0.950
XTrainPValue=XTrainPVal[:,[0,3,5]]
XTestPValue=XTestPVal[:,[0,3,5]]
regressorOLS=sm.OLS(exog=XTrainPValue,endog=YTrain).fit()
regressorOLS.summary()#Adj. R-squared:	0.951
XTrainPValue=XTrainPValue[:,1:3]
XTestPValue=XTestPValue[:,1:3]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.953
Model:                            OLS   Adj. R-squared:                  0.951
Method:                 Least Squares   F-statistic:                     426.3
Date:                Mon, 25 Nov 2019   Prob (F-statistic):           1.27e-28
Time:                        10:12:45   Log-Likelihood:                 3.7770
No. Observations:                  45   AIC:                            -1.554
Df Residuals:                      42   BIC:                             3.866
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0103      0.034     -0.301      0.765      -0.080       0.059
x1             0.9116      0.048     19.024      0.000       0.815       1.008
x2             0.1050      0.048      2.169      0.036       0.007       0.203
==============================================================================
Omnibus:                       13.304   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               16.275
Skew:                          -0.982   Prob(JB):                     0.000292
Kurtosis:                       5.195   Cond. No.                         2.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
######## Check VIF ##############
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrainPValue.shape[1]
for col in range(0,XTrainPValue.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrainPValue,exog_idx=col)
## All values are less than 10 so no need to reduce column
XTrainViffed=XTrainPValue
XTestViffed=XTestPValue
VIF

[2.0085577606815797, 2.0085577606815805]

In [0]:
########## Check if Linear or Non Linear #################
from sklearn.svm import SVR
RegressorSVR=SVR()
parameters=[{'kernel':['linear','rbf','poly','sigmoid']}]
from sklearn.model_selection import GridSearchCV
GS=GridSearchCV(estimator=RegressorSVR,param_grid=parameters,scoring='neg_mean_squared_error',cv=9)
GS.fit(XTrainViffed,YTrain)
best_params=GS.best_params_#linear
best_params

{'kernel': 'linear'}

In [0]:
######## Check Hyperparameters ###########
from sklearn.linear_model import LinearRegression
LR=LinearRegression()
hyperparameters=[{'fit_intercept':[True,False],
                  'normalize':[True,False]}]
GS_hyp=GridSearchCV(estimator=LR,param_grid=hyperparameters,scoring='neg_mean_squared_error',cv=9)
GS_hyp.fit(XTrainViffed,YTrain)
best_hyperparams=GS_hyp.best_params_#'fit_intercept': False, 'normalize': True

In [0]:
######## Create Regressor and ApplyKFold ##########
from sklearn.model_selection import KFold
from sklearn import metrics as mt
import statistics as st
RegressorLR=LinearRegression(fit_intercept=False,normalize=True)
r2=[]
kfold=KFold(n_splits=9,shuffle=True)
for count in range(0,100):
  for train_idx,val_idx in kfold.split(XTrainViffed):
    XTrainKF_Train,XTrainKF_Val=XTrainViffed[train_idx],XTrainViffed[val_idx]
    YTrainKF_Train,YTrainKF_Val=YTrain[train_idx],YTrain[val_idx]
    RegressorLR.fit(XTrainKF_Train,YTrainKF_Train)
    YPred=RegressorLR.predict(XTrainKF_Val)
    r2.append(mt.r2_score(YTrainKF_Val,YPred))
print("Goodness Of Fit in Training Set: "+str(st.mean(r2)))#Goodness Of Fit: 0.9043597819765842

Goodness Of Fit in Training Set: 0.9027389370382594


In [0]:
########## Predict ##################
RegressorLR.fit(XTrainViffed,YTrain)
YPred=sc_Y.inverse_transform(RegressorLR.predict(XTestViffed))
print("Goodness Of Fit in Test Set: "+str(mt.r2_score(sc_Y.inverse_transform(YTest),YPred)))#Goodness Of Fit:0.8784462981958525 

Goodness Of Fit in Test Set: 0.8784462981958525
